In [99]:
import pandas as pd
import numpy as np
import os
import json
# import gui
from collections import Counter
import re
import math as m

pd.options.display.max_columns = 20000

# Create a pandas dataframe of all cards
![Alt Text](https://media.giphy.com/media/vFKqnCdLPNOKc/giphy.gif)

### Create initial dataframe and perform some preprocessing tasks

In [100]:
lexicon = pd.read_feather('mtg_card_database')
colors = ["W", "U", "B", "R", "G", "C", "P",]

In [101]:
# Only keep the most recent arena set, dropping all reprints
lexicon = lexicon.sort_values(by='arena_id', ascending=False).copy()
lexicon = lexicon.drop_duplicates(subset=["name"])

# Set the index for card names to easily search
def set_index(lexicon):
    lexicon = lexicon.set_index("name", drop=False)
    lexicon.index.name = "card_name"
    return lexicon
lexicon = set_index(lexicon)

# Drop unused columns
lexicon = lexicon.drop(columns=['id', 'power', 'toughness', 'color_identity', 'set', 'keywords', 'flavor_text', 'edhrec_rank', 'arena_id', 'loyalty'], inplace=False)

In [102]:
# Clean mana_cost
def clean_mana_cost(s):
    """Cleans the mana_cost column by removing curly braces {} and by converting any numbers in mana cost to 1s representation

    Args:
        s (string): the mana_cost string 

    Returns:
        string: cleaned mana cost
    """
    # If s is not nonetype
    try: 
        s = s.replace("{", "").replace("}", "")

        # If the first character of s is a number
        try:
            num = int(s[0])
            s = s.replace(s[0], '1' * num)
        except:
            pass
    except:
        pass

    return s

lexicon['mana_cost'] = lexicon['mana_cost'].apply(lambda s: clean_mana_cost(s))

## Fix Dual-Faced Cards

### Extract card data for both sides

In [103]:
# Let's first create a dataframe that just has the card name and the column 'card_faces'
double_cards_df = lexicon[['name','card_faces']].dropna()

# DFC are a list of two dictionaries [ {k:v, k2:v2}, {k3:v3} ]
def split_card_faces(row, face_number):
    face = row[face_number]
    return face

face_one = 0
face_two = 1
double_cards_df['face1'] = double_cards_df['card_faces'].apply(lambda row: split_card_faces(row, face_one))
double_cards_df['face2'] = double_cards_df['card_faces'].apply(lambda row: split_card_faces(row, face_two))

# # Now let's drop the column 'card_faces'
# double_cards_df = double_cards_df.copy()
# double_cards_df.drop("card_faces",axis=1)

# We now go into each key within the dictionary of face1 and face2 and separate them into columns
try:
    double_cards_df[double_cards_df['face1'].apply(pd.Series).columns + "_1"] = double_cards_df['face1'].apply(pd.Series)
    double_cards_df[double_cards_df['face2'].apply(pd.Series).columns + "_2"] = double_cards_df['face2'].apply(pd.Series)
except:
    pass

# Define a list of columns we want to keep from the DFCs
cols_to_keep = ['name', 'name_1', 'name_2', 'oracle_text_1','oracle_text_2',
                'mana_cost_1', 'mana_cost_2', 'type_line_1', 'type_line_2']

# For each column in the dataframe, if it's not a selected column, we drop it
for col in double_cards_df.columns:
    if col not in cols_to_keep:
        double_cards_df.drop(col, axis=1, inplace=True)

# We now need to consolidate the 2 oracle texts into 1, we join them together
double_cards_df['oracle_text_dobles'] = double_cards_df['oracle_text_1'] + "\n" + double_cards_df['oracle_text_2']

# Clean the mana costs
double_cards_df['mana_cost_1'] = double_cards_df['mana_cost_1'].apply(lambda s: clean_mana_cost(s))
double_cards_df['mana_cost_2'] = double_cards_df['mana_cost_2'].apply(lambda s: clean_mana_cost(s))

In [104]:
# We now merge them by card name
lexicon = lexicon.merge(double_cards_df, on=["name"], how="left")

In [105]:
# If oracle_text is empty (meaning it's a double faced card), we replace it with our 'oracle_text_dobles' column
lexicon['oracle_text'] = np.where(lexicon['oracle_text'].isna(),lexicon['oracle_text_dobles'],lexicon['oracle_text'])


In [106]:
# Drop secret lair dual faced cards which should be single faced cards
tmpdf = lexicon[lexicon['name'].str.contains("//")]
mask = tmpdf[tmpdf['type_line'].isna()]
lexicon = lexicon[~lexicon['name'].isin(mask['name'])]
lexicon = lexicon.reset_index()

In [107]:
# We use this script to replace Nulls with "None"
# lexicon[['oracle_text_1','oracle_text_2']] = lexicon[['oracle_text_1','oracle_text_2']]

# try:
#     lexicon[['name', 'name_1', 'name_2', 'oracle_text_1','oracle_text_2',
#                 'mana_cost_1', 'mana_cost_2', 'type_line_1', 'type_line_2']] = lexicon[['name', 'name_1', 'name_2', 'oracle_text_1','oracle_text_2',
#                                                                                         'mana_cost_1', 'mana_cost_2', 'type_line_1', 'type_line_2']]
# except:
#     pass

# Now that we have our oracle text from the 2 card sides joined together, we want to use it to replace
# the actual "oracle_text" from the original dataframe, which is actually empty

# lexicon = set_index(lexicon)

# # And now that column is useless so we drop it
# lexicon = lexicon.drop("oracle_text_dobles",axis=1)

### Average the mana cost of both sides as well as the cmc

In [108]:
def dfc_avg_mana_cost_and_cmc(row):
    """Averages the mana cost for dual-faced cards and the cmc

    Args:
        row (pd.Series): one row of the lexicon df

    Returns:
        tuple of strings: averaged mana cost and average cmc
    """
    # Only for dual faced cards
    if isinstance(row.card_faces, type(np.ndarray(0))):
        # Initialize return value
        mana_cost = ''
        # Only for DFC with mana costs
        try:
            # Count the values of each mana symbol and add them together into a dictionary
            s1 = Counter(row.mana_cost_1)
            s2 = Counter(row.mana_cost_2)
            s3 = s1+s2
            if s3 == s1:
                row.mana_cost = row.mana_cost_1
                return row
            elif s3 == s2:
                row.mana_cost = row.mana_cost_2
                return row
            s4 = dict(s3)
            # Divide all the values by 2 rounded up for Mana symbols and down for generic costs
            for k in s4.keys():
                # If there is a generic casting cost
                try:
                    int(k)
                    s4[k] = max(m.floor(s4[k] / 2), 1)
                # If there is a colored mana symbol
                except:
                    s4[k] = m.ceil(s4[k] / 2)
                # concatenate the dict back into a string
                mana_cost += (k * s4[k])
            # Average cmc which is a return value
            avg_cmc = sum(s3.values()) / 2
            row.cmc = avg_cmc
            row.mana_cost = mana_cost
            # Return the averaged mana cost as a string
            return row
        except:
            pass
    # If its not a DFC then just return the mana_cost already present
    return row

lexicon = lexicon.apply(dfc_avg_mana_cost_and_cmc, axis=1)

### Create columns for casting cost pips
notes: 
- Phyrexian mana is represented as a P
- Hybrid mana counts as 1/2 for either color source


In [109]:
def count_pips(mana_cost):
    """Applied to the mana_cost column in lexicon to create columns for the number of pips in that color for each color

    Args:
        mana_cost (string): the mana cost as a str

    Returns:
        Counter: histogram of mana cost pips
    """
    pips = Counter(mana_cost)

    # Count hybrid mana as 1/2 a regular cost
    if '/' in pips:
        for i in range(len(mana_cost)):
            if mana_cost[i] == '/':
                try:
                    pips[mana_cost[i-1]] -= 0.5
                    pips[mana_cost[i+1]] -= 0.5
                except:
                    pass

        return pips

    return pips

lexicon = lexicon.copy()
for color in colors:
    row_name = f'cast_cost_{color}'
    lexicon.loc[:, f'{row_name}'] = lexicon['mana_cost'].apply(lambda row: count_pips(row)[f'{color}'])

### Create columns for produced mana colors of non-land cards

In [110]:
def count_produced_mana(produced_mana):
    """Applied to the produced_mana column, returns the colors produced by a card

    Args:
        row (List[str]): 

    Returns:
        Counter: a histogram of the produced mana symbols in the card
    """    
    return Counter(produced_mana)

lexicon = lexicon.copy()
for color in colors:
    row_name = f'produces_{color}'
    lexicon.loc[:, f'{row_name}'] = lexicon['produced_mana'].apply(lambda row: count_produced_mana(row)[f'{color}'])

## X costed spells
Count the number of X's in the spell and return that in a new column

In [111]:
def count_x_costs(mana_cost):
    """Creates a column that includes a count for the number of X's in the mana cost of a card

    Args:
        mana_cost (str): card's mana cost

    Returns:
        int: The number of X's
    """
    hist = Counter(mana_cost)
    num_x = hist['X']

    return num_x

lexicon = lexicon.copy()
lexicon.loc[:, 'num_x_in_mana_cost'] = lexicon['mana_cost'].apply(lambda row: count_x_costs(row))

## Card Selection

In [112]:
# Column for looting
lexicon['has_looting'] = np.where((
    (lexicon['oracle_text'].str.lower().str.contains(r"draw (a|one|two|three|four) (card|cards), then discard (a|one|two|three|four) (card|cards)",regex=True)==True) |
    (lexicon['oracle_text'].str.lower().str.contains(r"discard (a|one|two|three|four) (card|cards)(,|:) (draw|then draw) (a|one|two|three|four) (card|cards)",regex=True)==True) |
    (lexicon['oracle_text'].str.lower().str.contains(r"create (.*?) (blood|clue) token",regex=True)==True) |
    (lexicon['oracle_text'].str.lower().str.contains(r"cycling",regex=True)==True)
    ),1,0)

C:\Users\Peter\AppData\Local\Temp\ipykernel_18744\387049782.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (lexicon['oracle_text'].str.lower().str.contains(r"draw (a|one|two|three|four) (card|cards), then discard (a|one|two|three|four) (card|cards)",regex=True)==True) |
C:\Users\Peter\AppData\Local\Temp\ipykernel_18744\387049782.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (lexicon['oracle_text'].str.lower().str.contains(r"discard (a|one|two|three|four) (card|cards)(,|:) (draw|then draw) (a|one|two|three|four) (card|cards)",regex=True)==True) |
C:\Users\Peter\AppData\Local\Temp\ipykernel_18744\387049782.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (lexicon['oracle_text'].str.lower().str.contains(r"create 

In [113]:
# Column for card draw
lexicon['has_carddraw'] = np.where((
    (lexicon['oracle_text'].str.lower().str.contains(r"draw (a|one|two|three|four|five|six|seven|x|(.*?) x) (card|cards)", regex=True)==True) |
    (lexicon['oracle_text'].str.lower().str.contains(r"draw (cards equal to|that many cards)", regex=True)==True) |
    (lexicon['oracle_text'].str.lower().str.contains(r"target player draws (.*?) (card|cards)", regex=True)==True) |
    (lexicon['oracle_text'].str.lower().str.contains(r"(look at|reveal) the.*?put.*?(into|in) your hand", regex=True)==True) |
    (lexicon['oracle_text'].str.lower().str.contains(r"(exile|look at the).*?(card|cards).*?you may (cast|play)", regex=True)==True)
    ) & (
    (lexicon['oracle_text'].str.lower().str.contains(r"whenever you draw a card", regex=True)==False) &
    (lexicon['oracle_text'].str.lower().str.contains(r"if you would draw a card", regex=True)==False) &
    (lexicon['oracle_text'].str.lower().str.contains(r"draw (a|one|two|three|four) (card|cards), then discard (a|one|two|three|four) (card|cards)", regex=True)==False) &
    (lexicon['oracle_text'].str.lower().str.contains(r"discard (a|one|two|three|four) (card|cards), then draw (a|one|two|three|four) (card|cards)", regex=True)==False)
    ),1,0)

C:\Users\Peter\AppData\Local\Temp\ipykernel_18744\328286145.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (lexicon['oracle_text'].str.lower().str.contains(r"draw (a|one|two|three|four|five|six|seven|x|(.*?) x) (card|cards)", regex=True)==True) |
C:\Users\Peter\AppData\Local\Temp\ipykernel_18744\328286145.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (lexicon['oracle_text'].str.lower().str.contains(r"draw (cards equal to|that many cards)", regex=True)==True) |
C:\Users\Peter\AppData\Local\Temp\ipykernel_18744\328286145.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (lexicon['oracle_text'].str.lower().str.contains(r"target player draws (.*?) (card|cards)", regex=True)==True) |
C:\Users\Peter\AppData\Local\Tem

## Treasure Token Creation

In [114]:
regex_4 = "(a|one|up to one|two|up to two|three|up to three|four|up to four|five|up to five|six|up to six|x|up to x)"
# Makes treasures
lexicon['treasure_tokens'] = np.where((
    (lexicon['oracle_text'].str.lower().str.contains(f"(create|put) {regex_4}.*?treasure (token|tokens)", regex=True)==True)
    ),1,0)

C:\Users\Peter\AppData\Local\Temp\ipykernel_18744\1843537781.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (lexicon['oracle_text'].str.lower().str.contains(f"(create|put) {regex_4}.*?treasure (token|tokens)", regex=True)==True)


## Cost Reduction

In [135]:
lexicon['reduced_spells'] = np.where(
    (
    (lexicon['oracle_text'].str.lower().str.contains(r"(rather than pay) (its|it's|their|this spell's|the) mana cost", regex=True)==True) |
    (lexicon['oracle_text'].str.lower().str.contains(r"This spell costs \{[/WUBRGC]\}+|\{[1-9]\}\s\{[/WUBRGC]\}+|\{10\}\s\{[/WUBRG]\}+|\{X\}|\{X\}\s\{[/WUBRGC]}+ less to cast", regex=True)==True)) |
    (lexicon['oracle_text'].str.lower().str.contains(r"affinity for", regex=True)==True
    ),1,0)

C:\Users\Peter\AppData\Local\Temp\ipykernel_18744\3306278098.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (lexicon['oracle_text'].str.lower().str.contains(r"(rather than pay) (its|it's|their|this spell's|the) mana cost", regex=True)==True) |


In [137]:
lexicon['free_spells'] = np.where(
    (
    (lexicon['oracle_text'].str.lower().str.contains(r"(without|instead\sof paying) (its|it's|their|this spell's|the) mana cost", regex=True)==True)|
    ((lexicon['oracle_text'].str.lower().str.contains(r"cast|play")) & 
    (lexicon['oracle_text'].str.lower().str.contains(r"spell|spells|card|cards")) & 
    (lexicon['oracle_text'].str.lower().str.contains(r"without paying")) & 
    (lexicon['oracle_text'].str.lower().str.contains(r"mana")))
    ),1,0)

C:\Users\Peter\AppData\Local\Temp\ipykernel_18744\2658018507.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (lexicon['oracle_text'].str.lower().str.contains(r"(without|instead\sof paying) (its|it's|their|this spell's|the) mana cost", regex=True)==True)|


In [145]:
lexicon[lexicon['name'] == 'Reaper King']
# lexicon[lexicon['oracle_text'].str.contains(r"This spell costs \{X\} less to cast")]

,index,name,mana_cost,cmc,type_line,oracle_text,colors,produced_mana,rarity,card_faces,mana_cost_1,name_1,oracle_text_1,type_line_1,mana_cost_2,name_2,oracle_text_2,type_line_2,oracle_text_dobles,cast_cost_W,cast_cost_U,cast_cost_B,cast_cost_R,cast_cost_G,cast_cost_C,cast_cost_P,produces_W,produces_U,produces_B,produces_R,produces_G,produces_C,produces_P,num_x_in_mana_cost,has_looting,has_carddraw,treasure_tokens,reduced_spells,free_spells
12152,12154,Reaper King,11/W11/U11/B11/R11/G,10.0,Legendary Artifact Creature — Scarecrow,({2/W} can be paid with any two mana or with {...,"[B, G, R, U, W]",None,mythic,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,0.5,0.5,0.5,0.5,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [148]:
tmp = lexicon[lexicon['free_spells'] == 1]
tmp

,index,name,mana_cost,cmc,type_line,oracle_text,colors,produced_mana,rarity,card_faces,mana_cost_1,name_1,oracle_text_1,type_line_1,mana_cost_2,name_2,oracle_text_2,type_line_2,oracle_text_dobles,cast_cost_W,cast_cost_U,cast_cost_B,cast_cost_R,cast_cost_G,cast_cost_C,cast_cost_P,produces_W,produces_U,produces_B,produces_R,produces_G,produces_C,produces_P,num_x_in_mana_cost,has_looting,has_carddraw,treasure_tokens,reduced_spells,free_spells
66,66,Arcane Proxy,1111111,7.0,Artifact Creature — Wizard,Prototype {1}{U}{U} — 2/1 (You may cast this s...,[],None,mythic,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,1
388,388,"Gix, Yawgmoth Praetor",1BB,3.0,Legendary Creature — Phyrexian Praetor,Whenever a creature deals combat damage to one...,[B],None,mythic,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2.0,0.0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,1
419,419,One with the Multiverse,111111UU,8.0,Enchantment,You may look at the top card of your library a...,[U],None,mythic,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0,0.0,0.0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1
538,538,"Jodah, the Unifier",WUBRG,5.0,Legendary Creature — Human Wizard,"Legendary creatures you control get +X/+X, whe...","[B, G, R, U, W]",None,mythic,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,1
606,606,Keldon Flamesage,11R,3.0,Creature — Human Shaman,"Enlist\nWhenever Keldon Flamesage attacks, loo...",[R],None,rare,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26992,27003,Spellshift,111U,4.0,Instant,Counter target instant or sorcery spell. Its c...,[U],None,rare,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1
27083,27094,Roiling Horror,111BB,5.0,Creature — Horror,Roiling Horror's power and toughness are each ...,[B],None,rare,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2.0,0.0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1
27209,27220,Demonic Dread,1BR,3.0,Sorcery,"Cascade (When you cast this spell, exile cards...","[B, R]",None,common,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,1
27252,27263,Captured Sunlight,11GW,4.0,Sorcery,"Cascade (When you cast this spell, exile cards...","[G, W]",None,common,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,1.0,0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,1


~~- Mana_Costs
    - Phyrexian Mana
    - Dual Color (either red or white mana can be used to pay for RW)
    - X cost (average value for X is math.floor(4 / num_x))~~
    
~~- Card Selection
    - define RegEx pattern
    - Create method~~
~~- Cards which make treasure (RegEx pattern "Create a Treasure Token")~~

~~Cards which can be cast for free or which cast other cards for free~~
- Beautiful Soup
    - How to use it
    - How big do we want our dataset
    - How do we store the dataframes???

Could combine all card values into one so that one deck will be one row in the final dataframe?
- Might not be a good idea because where card selection and treasure is on the mana curve matters
- Also more color pips on cards that are low cmc are of higher importance